This Jupyter notebook follows the instruction given on https://spatialdata.scverse.org/en/stable/tutorials/notebooks/notebooks/examples/technology_visium_hd.html, which gives out instructions on how to plot Visium HD datasets using python.

Challenges on handling and plotting Visium HD data: \
The data is very sparse. In the dataset used in this notebook, only around 5.5 million of the bins are actually covering parts of the tissue where gene expression is detected. Even within these bins, most of the genes are not expressed. \
\
Raster data: not memory efficient. Fix: decouple spatial locations and gene expression.   
Spatial location: vector geometries. Gene expression info: AnnData table



In [1]:
# reload setup
%load_ext autoreload
%autoreload 2

Typically, we use spatialdata-io to read Visium HD data. Use spatialdata-io to convert to Zarr data. Downloaded (need work on understanding spatialdata-io and conversion to Zarr data)

In [7]:
# data preparatory work
# data already downloaded
visium_hd_zarr_path = "D:/Hicks Lab/visium_hd.zarr"

# read the zarr data format
import spatialdata as sd

sdata = sd.read_zarr(visium_hd_zarr_path)
sdata


c:\Users\danie\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\danie\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\danie\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


SpatialData object, with associated Zarr store: D:\Hicks Lab\visium_hd.zarr
├── Images
│     ├── 'Visium_HD_Mouse_Small_Intestine_cytassist_image': DataArray[cyx] (3, 3000, 3200)
│     ├── 'Visium_HD_Mouse_Small_Intestine_full_image': DataTree[cyx] (3, 21943, 23618), (3, 10971, 11809), (3, 5485, 5904), (3, 2742, 2952), (3, 1371, 1476)
│     ├── 'Visium_HD_Mouse_Small_Intestine_hires_image': DataArray[cyx] (3, 5575, 6000)
│     └── 'Visium_HD_Mouse_Small_Intestine_lowres_image': DataArray[cyx] (3, 558, 600)
├── Shapes
│     ├── 'Visium_HD_Mouse_Small_Intestine_square_002um': GeoDataFrame shape: (5479660, 1) (2D shapes)
│     ├── 'Visium_HD_Mouse_Small_Intestine_square_008um': GeoDataFrame shape: (351817, 1) (2D shapes)
│     └── 'Visium_HD_Mouse_Small_Intestine_square_016um': GeoDataFrame shape: (91033, 1) (2D shapes)
└── Tables
      ├── 'square_002um': AnnData (5479660, 19059)
      ├── 'square_008um': AnnData (351817, 19059)
      └── 'square_016um': AnnData (91033, 19059)
with coord

In [8]:
# let's make the var names unique; this improves performance in accessing the tabular data and is necessary to be able to plot the data
for table in sdata.tables.values():
    table.var_names_make_unique()

Spatial data object contains several images:  
1 large microscopy image, 2 downscaled version, and 1 CytAssist (for different workflow, used for overlaying data with histological features)  
Also, data at different resolutions (Ann Data)

**Plotting Images**

In [ ]:
# importing libraries
import matplotlib.pyplot as plt
import spatialdata_plot # used for rendering images method

axes = plt.subplots(1, 2, figsize=(10, 5))[1].flatten() # setting up plots

# render 2 images from the sdata object, and plot side by side
# render_images method is used (sdata.pl for plotting class)
sdata.pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.show(ax=axes[0], title="Full image")
sdata.pl.render_images("Visium_HD_Mouse_Small_Intestine_cytassist_image").pl.show(ax=axes[1], title="CytAssit image")

Issue: want to plot the same range.  
Solution: compute extent of full image, crop the second data (CytAssit) using query methods

In [ ]:
# calling get_extent from spatialdata to get the extent (limits) of full image
from spatialdata import get_extent

data_extent = get_extent(sdata["Visium_HD_Mouse_Small_Intestine_full_image"], coordinate_system="global")
data_extent

# crop the CytAssit image, using bounding_box_query class
from spatialdata import bounding_box_query

queried_cytassist = bounding_box_query(
    sdata["Visium_HD_Mouse_Small_Intestine_cytassist_image"], # refer to CytAssist image from sdata
    min_coordinate=[data_extent["x"][0], data_extent["y"][0]],
    max_coordinate=[data_extent["x"][1], data_extent["y"][1]],
    axes=("x", "y"), # query along x and y axes
    target_coordinate_system="global", 
)
sdata["queried_cytassist"] = queried_cytassist # store it in sdata object

In [ ]:
# plot again
axes = plt.subplots(1, 2, figsize=(10, 5))[1].flatten()
sdata.pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.show(ax=axes[0], title="Full image")
sdata.pl.render_images("queried_cytassist").pl.show(ax=axes[1], title="CytAssit image")

Visualize on a smaller region: full image shows better resolution

In [ ]:
axes = plt.subplots(1, 2, figsize=(10, 5))[1].flatten()
crop0 = lambda x: bounding_box_query(
    x, min_coordinate=[5000, 8000], max_coordinate=[10000, 13000], axes=("x", "y"), target_coordinate_system="global"
)
crop0(sdata).pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.show(
    ax=axes[0], title="Full image", coordinate_systems="global"
)
crop0(sdata).pl.render_images("queried_cytassist").pl.show(
    ax=axes[1], title="CytAssit image", coordinate_systems="global"
)

**Plotting Gene Expression Data**